In [1]:
!pip install hsbalance

In [1]:
import numpy as np
import hsbalance as hs

Using the Community license in this session. If you have a full Xpress license, first set the XPAUTH_PATH environment variable to the full path to your license file, xpauth.xpr, and then restart Python. If you want to use the FICO Community license and no longer want to see this message, set the XPAUTH_PATH environment variable to: /home/maged/anaconda3/envs/balance/lib/python3.8/site-packages/xpress/license/community-xpauth.xpr
NB: setting XPAUTH_PATH will also affect any other Xpress products installed on your system.


# Huang test_case

The configuration of the gas turbine we consider is shown in Fig. 1. This gas turbine has a mass of 727,000 Kg
and is 61.65 m long.
This gas turbine has three planes (BZ-A, BZ-C and BZ-E) available for balancing. However, in most cases, only
two planes (BZ-A and BZ-E) are used because of the difficulty of placing weights in plane BZ-C. The objective
of balancing here is to minimize the maximum vibration amplitude of the rotor at the measuring planes by using
balance weights placed in planes BZ-A and BZ-E.
For this case, the only available balance weight type for balance planes BZ-A and BZ-E is 142 grams. Each balance
plane also has a limited number of weight holes, as shown in Fig. 1 (Plane BZ-A has balance holes placed every 7.5
degrees, and plane BZ-E has balance holes placed every 5 degrees). It is possible to use all available balance holes.
However, there is one extra constraint: at most one weight can be put in each balance hole.


In [2]:
ALPHA_math=[
                            ['0.085@27', '0.05@82'],
                            ['0.053@57', '0.071@15']]

A_math=[
                            ['32@357'], 
                            ['105@346']]

Convert to complex numbers (cartesian) form

In [3]:
A = hs.convert_matrix_to_cart(A_math)
ALPHA = hs.convert_matrix_to_cart(ALPHA_math)
# A, ALPHA

Adding ALPHA

In [4]:
alpha = hs.Alpha()
alpha.add(direct_matrix=ALPHA)

In [5]:
alpha.check()

/home/maged/Turbomachinery-Rotors-Balancing/src/hsbalance/CI_matrix.py:108: UserWarning: 
  warnings.warn('\nWarning: Influence Matrix is asymmetrical!')

Influence Matrix is asymmetrical, check your data.

No ill conditioned planes --> ok


In [6]:
print(alpha)


++++++++++++++++++++++++++++++++++++++++
Influence Coefficient Matrix
++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++
Coefficient Values
               Plane 1       Plane 2
Sensor 1  0.085 @ 27.0   0.05 @ 82.0
Sensor 2  0.053 @ 57.0  0.071 @ 15.0
End of Coefficient Values
++++++++++++++++++++++++++++++++++++++++

                   


## Solving with Least squares:

In [7]:
 # Instantiate least square model

In [8]:
model_LeastSquares = hs.LeastSquares(A, alpha)

In [9]:
W_LeastSquares = model_LeastSquares.solve() #solve

In [10]:
hs.convert_matrix_to_math(W_LeastSquares)

array([['639.888 @ 73.8'],
       ['1122.814 @ 165.2']], dtype='<U16')

In [11]:
model_LeastSquares.rmse()

0.0

## Splitting plane BZ-A

* Holes available: we select angles from 30-150 to reduce calculation time, holes spaced by 7.5 degrees

In [12]:
split_BZA = model_LeastSquares.create_split()

In [13]:
angles = np.arange(30,150,7.5)

* Weights available is 142 gram (5 ounces) and only one weight per hole is allowed.
* We can also constrain the problem to 2000 grams per plane to avoid solution infiltration. 

In [14]:
split_BZA.split_setup(0, max_number_weights_per_hole=1, holes_available=[angles]
                         ,weights_available=[142], max_weight_per_plane=2000)

Benchmarking the split_solve method

In [15]:
%%timeit
split_BZA.split_solve()

49.2 ms ± 654 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Solving the split problem

In [16]:
split_BZA.split_solve()

array([[0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0.]])

Getting the results in pandas dataframe (including none zero values only)

In [17]:
split_BZA.results()

,37.5,52.5,75.0,97.5,105.0
weights_available,,,,,
142,1.0,1.0,1.0,1.0,1.0


df returns dataframe that describes the number of weights(index) for every hole (columns).  

Calculating the error weight (unbalance weight value) caused by splitting

In [18]:
split_BZA.error()

array([2.10034846])

Calculating the equivalent weight caused by splitting 

In [19]:
split_BZA.error(options='equ')

array('640.2 @ 73.6', dtype='<U12')

which is close to the weight generated by the model '639.888@73.8'

In [20]:
split_BZA.error(options='problem_error') # getting the problem accuracy

array([0.])

  - prob_error: As I tried to solve the problem using only free solvers, I used [`mip_cvxpy`][mip_cvxpy] package that uses `CBC` solver to solver mip problems. To get the most accurate results we should have turned the problem into mixed integer second order cone which is not supported by CBC, so instead I reformulate the objective function to be linear optimizing the max or real and imaginary part of the result weight error. The splitting error is returned then so the user can estimate.
  - I finally settled on using solver = `XPRESS` which is free with limited matrix value of 5000. I believe in practice this is satisfying for balancing problem.
      - [`FICO® Xpress`][xpress] solver is [fast and accurate](./test/benchmark_splitting_solver.md) as it allows quadratic solving for the MIP problem.
[xpress]: https://www.fico.com/en/products/fico-xpress-solver
[mip_cvxpy]: https://pypi.org/project/mip-cvxpy/

We can update the original W_leastSquares weights with the new split weight we have just created

In [21]:
split_BZA.update()

/home/maged/Turbomachinery-Rotors-Balancing/src/hsbalance/model.py:255: UserWarning: This will change your model optimum solution.Choose confirm=True
  warnings.warn('This will change your model optimum solution.'


array([[  178.48435221+614.4911797j ],
       [-1085.53108741+286.93648042j]])

In [22]:
hs.convert_matrix_to_math(W_LeastSquares)  # The new W_LeastSquares

array([['639.888 @ 73.8'],
       ['1122.814 @ 165.2']], dtype='<U16')

By default update will not do anything but warning you, as this will change the optimum solution

In [23]:
split_BZA.update(confirm=True)

array([[  180.56557795+614.20840222j],
       [-1085.53108741+286.93648042j]])

In [24]:
hs.convert_matrix_to_math(W_LeastSquares)  # The new W_LeastSquares

array([['640.2 @ 73.6'],
       ['1122.814 @ 165.2']], dtype='<U16')

In [25]:
model_LeastSquares.rmse()

0.1449

We can see the RMS error has increased from 0 to 0.145 in this problem due to splitting. 

Print a report of the model

In [26]:
print(model_LeastSquares.info())


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
MODEL
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
MODEL TYPE
LeastSquares
End of MODEL TYPE
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

                   
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
INFLUENCE COEFFICIENT MATRIX

++++++++++++++++++++++++++++++++++++++++
Influence Coefficient Matrix
++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++
Coefficient Values
               Plane 1       Plane 2
Sensor 1  0.085 @ 27.0   0.05 @ 82.0
Sensor 2  0.053 @ 57.0  0.071 @ 15.0
End of Coefficient Values
++++++++++++++++++++++++++++++++++++++++

                   
End of INFLUENCE COEFFICIENT MATRIX
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

                   
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
INITIAL VIBRATION
              Vibration
Se

## Splitting plane BZ-C

* Holes available: we select angles from 100-200 to reduce calculation time, holes spaced by 5 degrees

In [27]:
split_BZC = model_LeastSquares.create_split()

In [28]:
angles = np.arange(100, 200, 5)

* Weights available is 142 gram (5 ounces) and only one weight per hole is allowed.
* We can also constrain the problem to 2000 grams per plane to avoid solution infiltration. 

In [29]:
split_BZC.split_setup(1, max_number_weights_per_hole=1, holes_available=[angles]
                         ,weights_available=[142], max_weight_per_plane=2000)

Benchmarking the split_solve method

In [30]:
%%timeit
split_BZC.split_solve()

81.2 ms ± 384 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Solving the split problem

In [31]:
split_BZC.split_solve()

array([[1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1.,
        1., 1., 1., 1.]])

Getting the results in pandas dataframe (including none zero values only)

In [32]:
split_BZC.results()

,100,135,150,165,175,180,185,190,195
weights_available,,,,,,,,,
142,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


df returns dataframe that describes the number of weights(index) for every hole (columns).  

Calculating the error weight (unbalance weight value) caused by splitting

In [33]:
split_BZC.error()

array([1.63300893])

Calculating the equivalent weight caused by splitting 

In [34]:
split_BZC.error(options='equ')

array('1124.27 @ 165.2', dtype='<U15')

which is close to the weight generated by the model '1122.814@165.2'

In [35]:
split_BZC.error(options='problem_error') # getting the problem accuracy

array([1.87276186e-16])

  - prob_error: As I tried to solve the problem using only free solvers, I used [`mip_cvxpy`][mip_cvxpy] package that uses `CBC` solver to solver mip problems. To get the most accurate results we should have turned the problem into mixed integer second order cone which is not supported by CBC, so instead I reformulate the objective function to be linear optimizing the max or real and imaginary part of the result weight error. The splitting error is returned then so the user can estimate.
  - I finally settled on using solver = `XPRESS` which is free with limited matrix value of 5000. I believe in practice this is satisfying for balancing problem.
      - [`FICO® Xpress`][xpress] solver is [fast and accurate](./test/benchmark_splitting_solver.md) as it allows quadratic solving for the MIP problem.
[xpress]: https://www.fico.com/en/products/fico-xpress-solver
[mip_cvxpy]: https://pypi.org/project/mip-cvxpy/

We can update the original W_leastSquares weights with the new split weight we have just created

In [36]:
split_BZC.update()

/home/maged/Turbomachinery-Rotors-Balancing/src/hsbalance/model.py:255: UserWarning: This will change your model optimum solution.Choose confirm=True
  warnings.warn('This will change your model optimum solution.'


array([[  180.56557795+614.20840222j],
       [-1085.53108741+286.93648042j]])

In [37]:
hs.convert_matrix_to_math(W_LeastSquares)  # The new W_LeastSquares

array([['640.2 @ 73.6'],
       ['1122.814 @ 165.2']], dtype='<U16')

By default update will not do anything but warn you, as this will change the optimum solution

In [38]:
split_BZC.update(confirm=True)

array([[  180.56557795+614.20840222j],
       [-1087.12774135+286.59382263j]])

In [39]:
hs.convert_matrix_to_math(W_LeastSquares)  # The new W_LeastSquares

array([['640.2 @ 73.6'],
       ['1124.27 @ 165.2']], dtype='<U15')

In [40]:
model_LeastSquares.rmse()

0.1099

Print a report of the model

In [41]:
print(model_LeastSquares.info())


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
MODEL
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
MODEL TYPE
LeastSquares
End of MODEL TYPE
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

                   
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
INFLUENCE COEFFICIENT MATRIX

++++++++++++++++++++++++++++++++++++++++
Influence Coefficient Matrix
++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++
Coefficient Values
               Plane 1       Plane 2
Sensor 1  0.085 @ 27.0   0.05 @ 82.0
Sensor 2  0.053 @ 57.0  0.071 @ 15.0
End of Coefficient Values
++++++++++++++++++++++++++++++++++++++++

                   
End of INFLUENCE COEFFICIENT MATRIX
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

                   
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
INITIAL VIBRATION
              Vibration
Se

We can see the rms error has increased from 0 to 0.1099 in this problem due to splitting. 

In [42]:
model_LeastSquares.split_instance   # We can see the splitting instances that override the optimum solution

### Discussion
- The results in the Huang paper are different from mine. But I would like to further prove that the above calculations give even much lesser error.  
- The time is an issue in splitting, you should solve the problem first and try to select suitable angles around the original phase angle.